In [2]:
import pandas as pd
import numpy as np


from sklearn.preprocessing import  PolynomialFeatures
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, explained_variance_score

# linear regressions
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor

# non-linear regressions
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

import src.wrangle as wr
import src.clusters as cl

In [16]:
df = wr.get_zillow()

In [50]:
df.shape

(49380, 16)

In [17]:
df.columns

Index(['sqft', 'garage_sqft', 'lot_sqft', 'age', 'structure_price', 'price',
       'land_price', 'tax_amount', 'bath', 'beds', 'bed_bath_ratio', 'city_id',
       'zip', 'latitude', 'longitude', 'fireplace', 'garage', 'hottub_spa',
       'pool', 'county_land_code', 'county_number', 'county_name', 'logerror'],
      dtype='object')

In [134]:
features = ['garage_sqft', 'age','beds','garage','fireplace','bath','land_price',\
            'bed_bath_ratio','lot_sqft','tax_amount','hottub_spa', 'Orange',
               'Ventura', 'LA','logerror']

In [53]:
features_counties = ['garage_sqft', 'age','beds','garage','fireplace','bath','land_price',\
            'bed_bath_ratio','lot_sqft','price','tax_amount','hottub_spa', 'logerror']

In [48]:

features_clusters = ['garage_sqft', 'age','beds','garage','fireplace','bath','land_price',\
            'bed_bath_ratio','lot_sqft','tax_amount','hottub_spa', 'Orange',\
            'Ventura', 'LA','logerror']

features_counties = ['garage_sqft', 'age','beds','garage','fireplace','bath','land_price',\
            'bed_bath_ratio','lot_sqft','tax_amount','hottub_spa', 'logerror']

In [49]:
df.columns

Index(['garage_sqft', 'age', 'beds', 'garage', 'fireplace', 'bath',
       'land_price', 'bed_bath_ratio', 'lot_sqft', 'price', 'tax_amount',
       'hottub_spa', 'Orange', 'Ventura', 'LA', 'logerror'],
      dtype='object')

In [33]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27652 entries, 15546 to 19930
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   garage_sqft     27652 non-null  int64  
 1   age             27652 non-null  int64  
 2   beds            27652 non-null  uint8  
 3   garage          27652 non-null  uint8  
 4   fireplace       27652 non-null  uint8  
 5   bath            27652 non-null  uint8  
 6   land_price      27652 non-null  int64  
 7   bed_bath_ratio  27652 non-null  float64
 8   lot_sqft        27652 non-null  int64  
 9   price           27652 non-null  int64  
 10  tax_amount      27652 non-null  int64  
 11  hottub_spa      27652 non-null  uint8  
 12  Orange          27652 non-null  int64  
 13  Ventura         27652 non-null  int64  
 14  LA              27652 non-null  int64  
dtypes: float64(1), int64(9), uint8(5)
memory usage: 2.5 MB


In [22]:
wr.print_value_counts(df)

garage_sqft


,index,garage_sqft
0,0,32382
1,400,755
2,441,664
3,462,643
4,360,618
...,...,...
773,1681,1
774,1090,1
775,1296,1
776,221,1



age


,index,age
0,62,1830
1,67,1561
2,63,1467
3,64,1445
4,61,1325
...,...,...
128,135,1
129,123,1
130,1,1
131,124,1



beds


...

,index,hottub_spa
0,0,48021
1,1,1359



county_name


,index,county_name
0,LA,21061
1,Orange,13205
2,LA_city,10879
3,Ventura,4235



logerror


,index,logerror
0,0.000231,5
1,0.002652,5
2,0.000727,5
3,0.001878,5
4,0.003152,4
...,...,...
49133,-0.015346,1
49134,-0.001411,1
49135,0.006841,1
49136,-0.067485,1


In [31]:
X_train, X_validate, X_test, y_train, y_validate, y_test = wr.full_split_zillow(df)

In [59]:
columns_to_scale = ['garage_sqft','age','beds','garage','fireplace','bath',\
 'land_price','bed_bath_ratio','lot_sqft','price','tax_amount']

['garage_sqft',
 'age',
 'beds',
 'garage',
 'fireplace',
 'bath',
 'land_price',
 'bed_bath_ratio',
 'lot_sqft',
 'price',
 'tax_amount']

In [24]:
df.county_name.unique()

array(['Orange', 'Ventura', 'LA', 'LA_city'], dtype=object)

In [26]:
def dummies(df):
    '''
    create dummy variables for LA and Ventura
    '''
    # create dummies for LA and Ventura
    df['Orange'] = np.where(df.county_name == 'Orange', 1, 0)
    df['Ventura'] = np.where(df.county_name == 'Ventura', 1, 0)
    df['LA'] = np.where(df.county_name == 'LA', 1, 0)
    return df.drop(columns='county_name')

In [37]:
df = wr.get_zillow()

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49380 entries, 0 to 52318
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   sqft              49380 non-null  int64  
 1   garage_sqft       49380 non-null  int64  
 2   lot_sqft          49380 non-null  int64  
 3   age               49380 non-null  int64  
 4   structure_price   49380 non-null  int64  
 5   price             49380 non-null  int64  
 6   land_price        49380 non-null  int64  
 7   tax_amount        49380 non-null  int64  
 8   bath              49380 non-null  uint8  
 9   beds              49380 non-null  uint8  
 10  bed_bath_ratio    49380 non-null  float64
 11  city_id           49380 non-null  int64  
 12  zip               49380 non-null  int64  
 13  latitude          49380 non-null  int64  
 14  longitude         49380 non-null  int64  
 15  fireplace         49380 non-null  uint8  
 16  garage            49380 non-null  uint8 

In [42]:
df = dummies(df)

In [39]:
la_city = df[df.county_name == 'LA_city'] # LA city
la = df[df.county_name == 'LA'] # LA county
ventura = df[df.county_name == 'Ventura'] # Ventura county
orange = df[df.county_name == 'Orange'] # Orange county

In [54]:
la_city = la_city[features_counties]
la = la[features_counties]
ventura = ventura[features_counties]
orange = orange[features_counties]

In [57]:
la_city.head(1)

,garage_sqft,age,beds,garage,fireplace,bath,land_price,bed_bath_ratio,lot_sqft,price,tax_amount,hottub_spa,logerror
14,0,109,2,0,0,1,134175,2.0,5091,171838,2220,0,-0.120976


In [56]:
X_train.head(1)

,garage_sqft,age,beds,garage,fireplace,bath,land_price,bed_bath_ratio,lot_sqft,price,tax_amount,hottub_spa,Orange,Ventura,LA
15546,0,36,5,0,0,3,250857,1.67,10243,610118,6827,0,0,0,1


In [95]:
seed = 42
# get a baseline value = median of the train set's target
baseline = y_train.mean()


# create dataframes to keep predictions of train and validate data sets
predictions_train = pd.DataFrame(y_train)
predictions_validate = pd.DataFrame(y_validate)
predictions_train['baseline'] = baseline
predictions_validate['baseline'] = baseline

###### GLOBAL EVALUATION VARS ##########
# calculate baseline RMSE score
RMSE_baseline = round(mean_squared_error(y_train, predictions_train.baseline) ** .5)
# DataFrame to keep model's evaluations
scores = pd.DataFrame(columns=['model_name', 'features', 'RMSE_train', 'R2_train', 'RMSE_validate', 
                               'R2_validate', 'RMSE_difference'])

# create a dictionary of regression models
models = {
    'Linear Regression': LinearRegression(),
    #'Generalized Linear Model': TweedieRegressor(power=2, alpha = 0.5),
    'Gradient Boosting Regression': GradientBoostingRegressor(random_state=seed),
    'Decision Tree Regression': DecisionTreeRegressor(max_depth=4, random_state=seed),
    'Random Forest Regression':RandomForestRegressor(max_depth=4, random_state=seed),
    'LassoLars Regression':LassoLars(alpha=0.1)
    }

In [77]:
features_poly = X_train_loc.iloc[:, :11].columns.tolist()

In [112]:
scores.index

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64')

In [115]:
scores.drop?

In [117]:
scores.drop(scores.index, axis=0, inplace=True)

,model_name,features,RMSE_train,R2_train,RMSE_validate,R2_validate,RMSE_difference


In [87]:
def regression_errors(y_actual, y_predicted):
    '''
    this function accepts 
    y: actual results/array
    yhat: predictions/array
    k: feature size/integer
    calculates regression scores based on the baseline being median
    returns RMSE and adjacted R2
    '''
    # root mean squared error score
    RMSE = mean_squared_error(y_actual, y_predicted) ** .5
    # adjucted R^2 score
    ADJR2 = explained_variance_score(y_actual, y_predicted)
    return round(RMSE), round(ADJR2, 2)

In [92]:
X_train.iloc[:, :6]

,garage_sqft,age,beds,garage,fireplace,bath
15546,-0.684051,-0.794326,1.941252,-0.693275,-0.375874,0.898908
50614,-0.684051,1.729279,-1.451389,-0.693275,-0.375874,-1.330084
46289,-0.684051,0.622435,0.810371,-0.693275,-0.375874,-0.215588
39699,-0.684051,0.533887,-0.320509,-0.693275,-0.375874,-0.215588
29962,-0.684051,-0.528684,0.810371,-0.693275,-0.375874,0.898908
...,...,...,...,...,...,...
47297,-0.684051,1.773553,0.810371,-0.693275,-0.375874,-0.215588
19369,0.683355,0.578161,-0.320509,0.341511,-0.375874,-0.215588
23744,-0.684051,0.622435,0.810371,-0.693275,-0.375874,-0.215588
49856,1.065424,-2.211087,-0.320509,1.376296,-0.375874,-0.215588


In [98]:
baseline

0.011098433054631289

In [101]:
scores

,model_name,features,RMSE_train,R2_train,RMSE_validate,R2_validate,RMSE_difference
0,Linear Regression,polynomial,0,0.01,0,0.00,0
1,Gradient Boosting Regression,polynomial,0,0.04,0,-0.00,0
2,Decision Tree Regression,polynomial,0,0.01,0,-0.01,0
3,Random Forest Regression,polynomial,0,0.02,0,0.00,0
4,LassoLars Regression,polynomial,0,0.00,0,0.00,0
5,Linear Regression,standard,0,0.01,0,0.00,0
6,Gradient Boosting Regression,standard,0,0.04,0,-0.00,0
7,Decision Tree Regression,standard,0,0.01,0,-0.00,0
8,Random Forest Regression,standard,0,0.02,0,0.00,0
9,LassoLars Regression,standard,0,0.00,0,0.00,0


In [96]:
run_polinomial(X_train.iloc[:, :-1], X_validate.iloc[:, :-1])

In [125]:
def run_polinomial(X1, X2, i=6, f_number=0):
    i = 6 # first 6 features
    poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
    poly.fit(X1.iloc[:, :i])
    # create a df with transformed features of the train set
    X1_poly = pd.DataFrame(
        poly.transform(X1.iloc[:, :i]),
        columns=poly.get_feature_names(X1.iloc[:, :i].columns),
        index=X1.index)
    X1_poly = pd.concat([X1_poly, X1.iloc[:, i:]], axis=1)
    #X1_poly = pd.concat([X1_poly, X1], axis=1)

    #display(X1_poly.head(1)) #testing the columns

    # create a df with transformed features for the validate set
    X2_poly = pd.DataFrame(
        poly.transform(X2.iloc[:, :i]),
        columns=poly.get_feature_names(X2.iloc[:, :i].columns),
        index=X2.index)
    X2_poly = pd.concat([X2_poly, X2.iloc[:, i:]], axis=1)
    #X2_poly = pd.concat([X2_poly, X2], axis=1)

    feature_name = 'poly'+ str(f_number)

    for key in models:
        # create a model
        model = models[key]
        # fit the model
        model.fit(X1_poly, y_train)
        # predictions of the train set
        y_hat_train = model.predict(X1_poly)
        # predictions of the validate set
        y_hat_validate = model.predict(X2_poly)
        # add train set predictions to the data frame
        predictions_train[key] = y_hat_train
        # add validate set predictions to the data frame
        predictions_validate[key] = y_hat_validate

        # calculate scores train set
        RMSE, R2 = regression_errors(y_train, y_hat_train)
        # calculate scores validation set
        RMSE_val, R2_val = regression_errors(y_validate, y_hat_validate)
        diff = np.abs(RMSE - RMSE_val)

        # add the score results to the scores Data Frame
        scores.loc[len(scores.index)] = [key, feature_name, RMSE, R2, RMSE_val, R2_val, diff]

In [126]:
def run_models(X_train, X_validate, f_number=0):
    
    '''
    general function to run models with X_train and X_validate that were scaled
    '''
    feature_name = 'stand ' + str(f_number)
    for key in models:
        # create a model
        model = models[key]
        # fit the model
        model.fit(X_train, y_train)
        # predictions of the train set
        y_hat_train = model.predict(X_train)
        # predictions of the validate set
        y_hat_validate = model.predict(X_validate)
        # add train set predictions to the data frame
        predictions_train[key] = y_hat_train
        # add validate set predictions to the data frame
        predictions_validate[key] = y_hat_validate

        # calculate scores train set
        RMSE, R2 = regression_errors(y_train, y_hat_train)
        # calculate scores validation set
        RMSE_val, R2_val = regression_errors(y_validate, y_hat_validate)
        diff = np.abs(RMSE - RMSE_val)
        
        # add the score results to the scores Data Frame
        scores.loc[len(scores.index)] = [key, feature_name, RMSE, R2, RMSE_val, R2_val, diff]

In [104]:
def run_all_models():
    '''
    the function runs all models and saves the results to csv file
    '''
    run_model_standard()
    run_polynomial()
    
    #scores.to_csv('regression_results.csv')

In [61]:
from importlib import reload

In [63]:
reload(wr)

<module 'src.wrangle' from '/Users/nadina/codeup-data-science/clustering-project/src/wrangle.py'>

In [64]:
X_train, X_validate, X_test = wr.standard_scale_zillow(X_train, X_validate, X_test)

In [106]:
X_train_loc.location_clusters.unique()

array([2, 5, 3, 1, 4, 0], dtype=uint8)

In [66]:
reload(cl)

<module 'src.clusters' from '/Users/nadina/codeup-data-science/clustering-project/src/clusters.py'>

In [68]:
X_train_loc, X_validate_loc, X_test_loc = cl.add_location_clusters(X_train, X_validate, X_test)

In [100]:
run_models(X_train, X_validate)

In [118]:
X_train_loc.columns

Index(['garage_sqft', 'age', 'beds', 'garage', 'fireplace', 'bath',
       'land_price', 'bed_bath_ratio', 'lot_sqft', 'price', 'tax_amount',
       'hottub_spa', 'Orange', 'Ventura', 'LA', 'location_clusters'],
      dtype='object')

In [146]:
def regression_errors(y_actual, y_predicted):
    '''
    this function accepts 
    y: actual results/array
    yhat: predictions/array
    k: feature size/integer
    calculates regression scores based on the baseline being median
    returns RMSE and adjacted R2
    '''
    # root mean squared error score
    RMSE = mean_squared_error(y_actual, y_predicted) ** .5
    # adjucted R^2 score
    ADJR2 = explained_variance_score(y_actual, y_predicted)
    return round(RMSE), round(ADJR2, 2)

In [147]:
def run_models(X_train, X_validate, y_train, y_validate, f_number):
    
    '''
    general function to run models with X_train and X_validate that were scaled
    '''
    feature_name = 'stand ' + str(f_number)
    for key in models:
        # create a model
        model = models[key]
        # fit the model
        model.fit(X_train, y_train)
        # predictions of the train set
        y_hat_train = model.predict(X_train)
        # predictions of the validate set
        y_hat_validate = model.predict(X_validate)


        # calculate scores train set
        RMSE, R2 = regression_errors(y_train, y_hat_train)
        # calculate scores validation set
        RMSE_val, R2_val = regression_errors(y_validate, y_hat_validate)
        diff = np.abs(RMSE - RMSE_val)
        
        # add the score results to the scores Data Frame
        scores.loc[len(scores.index)] = [key, feature_name, RMSE, R2, RMSE_val, R2_val, diff]

In [ ]:
def run_polinomial(X1, X2):
    i = 5 # first 6 features
    poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
    poly.fit(X1.iloc[:, :i])
    # create a df with transformed features of the train set
    X1_poly = pd.DataFrame(
        poly.transform(X1.iloc[:, :i]),
        columns=poly.get_feature_names(X1.iloc[:, :i].columns),
        index=X1.index)
    X1_poly = pd.concat([X1_poly, X1.iloc[:, i:]], axis=1)
    #X1_poly = pd.concat([X1_poly, X1], axis=1)

    #display(X1_poly.head(1)) #testing the columns

    # create a df with transformed features for the validate set
    X2_poly = pd.DataFrame(
        poly.transform(X2.iloc[:, :i]),
        columns=poly.get_feature_names(X2.iloc[:, :i].columns),
        index=X2.index)
    X2_poly = pd.concat([X2_poly, X2.iloc[:, i:]], axis=1)
    #X2_poly = pd.concat([X2_poly, X2], axis=1)

    feature_name = 'poly'#+ str(f_number)

    for key in models:
        # create a model
        model = models[key]
        # fit the model
        model.fit(X1_poly, y_train)
        # predictions of the train set
        y_hat_train = model.predict(X1_poly)
        # predictions of the validate set
        y_hat_validate = model.predict(X2_poly)

        # calculate scores train set
        RMSE, R2 = regression_errors(y_train, y_hat_train)
        # calculate scores validation set
        RMSE_val, R2_val = regression_errors(y_validate, y_hat_validate)
        diff = np.abs(RMSE - RMSE_val)

        # add the score results to the scores Data Frame
        scores.loc[len(scores.index)] = [key, feature_name, RMSE, R2, RMSE_val, R2_val, diff]

In [148]:
reload(md)

<module 'src.models' from '/Users/nadina/codeup-data-science/clustering-project/src/models.py'>

In [141]:
import src.models as md

In [149]:
md.get_scores()

,model_name,feature_name,RMSE_train,R2_train,RMSE_validate,R2_validate,RMSE_difference
0,Linear Regression,stand 0,0,0.00,0,0.00,0
1,Gradient Boosting Regression,stand 0,0,0.04,0,-0.00,0
2,Decision Tree Regression,stand 0,0,0.01,0,-0.02,0
3,Random Forest Regression,stand 0,0,0.02,0,0.00,0
4,LassoLars Regression,stand 0,0,0.00,0,0.00,0
5,Linear Regression,poly,0,0.00,0,0.00,0
6,Gradient Boosting Regression,poly,0,0.04,0,-0.00,0
7,Decision Tree Regression,poly,0,0.01,0,-0.00,0
8,Random Forest Regression,poly,0,0.02,0,0.00,0
9,LassoLars Regression,poly,0,0.00,0,0.00,0
